In [ ]:
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
from huggingface_hub import login



In [ ]:
class EmbeddingCalculator:
    """
    Calculates embeddings for text using a Hugging Face model.
    """
    def __init__(self, model_name, hf_token):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
        self.model = AutoModel.from_pretrained(
            model_name,
            use_auth_token=hf_token,
            device_map="auto",
            torch_dtype=torch.float16
        )

    def calculate_embedding(self, text):
        """
        Calculate the embedding for the given text.
        """
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {key: value.to(self.model.device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
            embedding = outputs.hidden_states[-1][:, 0, :].squeeze().cpu().numpy()
        return embedding



In [ ]:

class EnhancedDynamicMemory:
    """
    Stores embeddings dynamically and allows similarity-based retrieval.
    """
    def __init__(self):
        self.memory = {}

    def add_embedding(self, key, embedding, weight=1.0):
        """
        Add or update an embedding in the dynamic memory.
        Combines embeddings if the key already exists (weighted average).
        """
        if key in self.memory:
            # Combine existing and new embeddings
            old_embedding = self.memory[key]["embedding"]
            old_weight = self.memory[key]["weight"]
            combined_embedding = (old_embedding * old_weight + embedding * weight) / (old_weight + weight)
            self.memory[key] = {"embedding": combined_embedding, "weight": old_weight + weight}
        else:
            self.memory[key] = {"embedding": embedding, "weight": weight}

    def find_closest_embedding(self, embedding, threshold=0.8):
        """
        Find the closest embedding in memory using cosine similarity.
        Returns the key and similarity score if above the threshold, otherwise None.
        """
        max_similarity = 0
        closest_key = None

        for key, value in self.memory.items():
            stored_embedding = value["embedding"]
            similarity = cosine_similarity([embedding], [stored_embedding])[0][0]
            if similarity > max_similarity:
                max_similarity = similarity
                closest_key = key

        return closest_key, max_similarity if max_similarity >= threshold else None

    def visualize_memory(self):
        """
        Display the keys and weights of stored embeddings.
        """
        for key, value in self.memory.items():
            print(f"Key: {key}, Weight: {value['weight']}")


In [ ]:
class NextQueryPredictor:
    """
    Predicts the next queries using a Hugging Face causal language model.
    """
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16
        )

    def predict_next_queries(self, context, num_queries=5):
        """
        Predict the next queries based on the given context.
        """
        prompt = f"""
You are tasked with predicting the next {num_queries} queries based on the following context:
{context}

Provide the next {num_queries} queries as a numbered list:
"""
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        inputs = {key: value.to(self.model.device) for key, value in inputs.items()}

        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=300,
            temperature=0.7,
            do_sample=True,
            num_return_sequences=1
        )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self._extract_numbered_list(response)

    def _extract_numbered_list(self, response):
        """
        Extract a numbered list from the model's response.
        """
        import re
        numbered_lines = re.findall(r"^\d+\..*", response, re.MULTILINE)
        return [line.split(". ", 1)[1] for line in numbered_lines]

In [ ]:
def calculate_hits_and_accuracy(user_queries, predicted_queries_list):
    """
    Calculate hits, misses, and accuracy based on user queries and predicted queries.
    """
    hits = 0
    misses = 0

    for i, user_query in enumerate(user_queries):
        if i < len(predicted_queries_list):
            predicted_queries = predicted_queries_list[i]
            if user_query in predicted_queries:
                hits += 1
            else:
                misses += 1

    total_queries = len(user_queries)
    accuracy = (hits / total_queries) * 100 if total_queries > 0 else 0

    return hits, misses, accuracy

In [ ]:
def dynamic_teacher_student_logic(user_query, embedding_calculator, memory, context):
    """
    Implements dynamic teacher-student interaction with hit-or-miss logic.
    """
    # Calculate embedding for the new query
    new_embedding = embedding_calculator.calculate_embedding(user_query)

    # Check for a hit or miss in memory
    closest_key, similarity = memory.find_closest_embedding(new_embedding)

    if closest_key:
        print(f"Hit: Found similar query with similarity {similarity:.2f}")
        print(f"Closest Query: {closest_key}")
        # Refine embedding for this query
        memory.add_embedding(closest_key, new_embedding)
    else:
        print("Miss: No similar query found. Adding new query to memory.")
        memory.add_embedding(user_query, new_embedding)

    # Update the context with the user's query and a placeholder answer
    context += f"Q: {user_query}\nA: (Generated Answer)\n"
    return context

In [ ]:
def main():
    # Authenticate with Hugging Face
    print("Authenticating with Hugging Face...")
    login()
    hf_token = input("Enter your Hugging Face token: ").strip()

    # Model names
    embedding_model_name = "meta-llama/Llama-3.2-3B-Instruct"
    query_predictor_model_name = "meta-llama/Llama-3.2-3B-Instruct"

    # Initialize components
    embedding_calculator = EmbeddingCalculator(embedding_model_name, hf_token)
    memory = EnhancedDynamicMemory()
    query_predictor = NextQueryPredictor(query_predictor_model_name)

    # Initialize tracking variables
    user_queries = []
    predicted_queries_list = []
    context = ""  # Context for predictions

    print("\n=== Interactive Q&A Session ===")
    print("Ask your questions to the system.")
    print("Type 'exit' to end the session.\n")

    while True:
        # Get user input (actual query)
        user_query = input("You: ").strip()
        if user_query.lower() == "exit":
            break

        # Append the user query to the tracking list
        user_queries.append(user_query)

        # Generate predicted queries in the background
        predicted_queries = query_predictor.predict_next_queries(context)
        predicted_queries_list.append(predicted_queries)

        # Process the user query with teacher-student logic
        context = dynamic_teacher_student_logic(user_query, embedding_calculator, memory, context)

        # Simulate system's response
        print("System: (Generated Answer)")

    # Calculate hits, misses, and accuracy
    hits, misses, accuracy = calculate_hits_and_accuracy(user_queries, predicted_queries_list)

    print("\n=== Session Summary ===")
    print(f"Total User Queries: {len(user_queries)}")
    print(f"Hits: {hits}")
    print(f"Misses: {misses}")
    print(f"Accuracy: {accuracy:.2f}%")

    # Visualize memory
    print("\nMemory Contents:")
    memory.visualize_memory()


if __name__ == "__main__":
    main()